# Init

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import pdb

In [ ]:
env = UnityEnvironment(file_name="./Tennis.app")

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# Training

In [ ]:
env.close()